In [38]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score

In [39]:
iris_df = load_iris(as_frame=True)
X_df = iris_df.data
y_df = iris_df.target

In [40]:
X_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [41]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42, stratify=y_df)

In [45]:
models = {
    'LogisticRegression': LogisticRegression(max_iter=500),
    'RandomForest': RandomForestClassifier(),
    'SVC': SVC()
}

param_grids = {
    'LogisticRegression': {
        'C': [0.01, 0.1, 1, 10],
        'solver': ['lbfgs', 'liblinear']
    },
    'RandomForest': {
        'n_estimators': [10, 50, 100, 200],
        'max_depth': [None, 5, 10],
        'min_samples_split': [2, 5]
    },
    'SVC': {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf', 'poly'],
        'gamma': [0.1, 1, 'scale', 'auto']
    }
}


In [46]:
results = []

for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=5, scoring='f1_macro')
    print(f'{name} mean CV F1-score: {scores.mean():.2f}')
    # Grid Search
    grid = GridSearchCV(model, param_grids[name], cv=5, scoring='f1_macro', n_jobs=-1)
    grid.fit(X_train, y_train)
    y_pred_grid = grid.best_estimator_.predict(X_test)
    
    # Randomized Search
    random = RandomizedSearchCV(model, param_distributions=param_grids[name], n_iter=5, cv=5, scoring='f1_macro', n_jobs=-1, random_state=42)
    random.fit(X_train, y_train)
    y_pred_random = random.best_estimator_.predict(X_test)
    
    # Store results
    results.append({
        'Model': name,
        'Method': 'GridSearchCV',
        'Best Params': grid.best_params_,
        'Accuracy': accuracy_score(y_test, y_pred_grid),
        'Precision': precision_score(y_test, y_pred_grid, average='macro'),
        'Recall': recall_score(y_test, y_pred_grid, average='macro'),
        'F1-Score': f1_score(y_test, y_pred_grid, average='macro')
    })
    results.append({
        'Model': name,
        'Method': 'RandomizedSearchCV',
        'Best Params': random.best_params_,
        'Accuracy': accuracy_score(y_test, y_pred_random),
        'Precision': precision_score(y_test, y_pred_random, average='macro'),
        'Recall': recall_score(y_test, y_pred_random, average='macro'),
        'F1-Score': f1_score(y_test, y_pred_random, average='macro')
    })


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

LogisticRegression mean CV F1-score: 0.88
RandomForest mean CV F1-score: 0.96
SVC mean CV F1-score: 0.86


In [47]:
results_df = pd.DataFrame(results)
print(results_df[['Model', 'Method', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Best Params']])


                Model              Method  Accuracy  Precision    Recall  \
0  LogisticRegression        GridSearchCV  0.966667   0.969697  0.966667   
1  LogisticRegression  RandomizedSearchCV  0.966667   0.969697  0.966667   
2        RandomForest        GridSearchCV  0.966667   0.969697  0.966667   
3        RandomForest  RandomizedSearchCV  0.933333   0.933333  0.933333   
4                 SVC        GridSearchCV  0.966667   0.969697  0.966667   
5                 SVC  RandomizedSearchCV  0.933333   0.933333  0.933333   

   F1-Score                                        Best Params  
0  0.966583                        {'C': 1, 'solver': 'lbfgs'}  
1  0.966583                      {'solver': 'lbfgs', 'C': 0.1}  
2  0.966583  {'max_depth': None, 'min_samples_split': 5, 'n...  
3  0.933333  {'n_estimators': 10, 'min_samples_split': 2, '...  
4  0.966583       {'C': 0.1, 'gamma': 0.1, 'kernel': 'linear'}  
5  0.933333            {'kernel': 'rbf', 'gamma': 0.1, 'C': 1}  


In [48]:
results_df

,Model,Method,Best Params,Accuracy,Precision,Recall,F1-Score
0,LogisticRegression,GridSearchCV,"{'C': 1, 'solver': 'lbfgs'}",0.966667,0.969697,0.966667,0.966583
1,LogisticRegression,RandomizedSearchCV,"{'solver': 'lbfgs', 'C': 0.1}",0.966667,0.969697,0.966667,0.966583
2,RandomForest,GridSearchCV,"{'max_depth': None, 'min_samples_split': 5, 'n...",0.966667,0.969697,0.966667,0.966583
3,RandomForest,RandomizedSearchCV,"{'n_estimators': 10, 'min_samples_split': 2, '...",0.933333,0.933333,0.933333,0.933333
4,SVC,GridSearchCV,"{'C': 0.1, 'gamma': 0.1, 'kernel': 'linear'}",0.966667,0.969697,0.966667,0.966583
5,SVC,RandomizedSearchCV,"{'kernel': 'rbf', 'gamma': 0.1, 'C': 1}",0.933333,0.933333,0.933333,0.933333


In [49]:
best_row = results_df.iloc[results_df['F1-Score'].idxmax()]
print("\nBest Model:")
print(best_row)



Best Model:
Model                   LogisticRegression
Method                        GridSearchCV
Best Params    {'C': 1, 'solver': 'lbfgs'}
Accuracy                          0.966667
Precision                         0.969697
Recall                            0.966667
F1-Score                          0.966583
Name: 0, dtype: object
